# Reproducing de Boer's results (2000)

As a first step, we try to reproduce the results found by de Boer in his paper from 2000.
This paper will form the basis for our project thus reproducing its results is an important first step.
Thanks to de Boer being the teaching professor of the Evolution of Speech course @ VUB 2021-2022, we have access to the source code of this paper.

**Reference**: de Boer, B. (2000). Self-organization in vowel systems. *In Journal of Phonetics (Vol. 28, Issue 4, pp. 441–465)*. Elsevier BV. https://doi.org/10.1006/jpho.2000.0125

## Table of contents

- Student information
- Synthesizer equations

## Student information

- **Name**: Bontinck Lennert
- **Student ID**: 568702
- **Affiliation**: VUB - Master Computer Science: AI

## Synthesizer equations

Bart de Boer's system consists of agents who can produce, perceive, and remember speech sounds in a human-like way.
To make this work the agents are equipped with an articulatory synthesizer.
This synthesizer takes as input the three major vowel parameters:

- tongue position: **p**
- tongue height: **h**
- lip rounding: **r**

The outputs of the synthesizer are the first four formant frequencies of the corresponding vowel.

- $F_1 (Hz)$
- $F_2 (Hz)$
- $F_3 (Hz)$
- $F_4 (Hz)$

Example input/outputs are given in table 1 of de Boer's paper (2000) and the Equations for $F_1$ to $F_4$ are given in table 1 of his paper (2000).
In the below code block we will first make the functions for each $F_x$ and check their output by comparing the values to the examples from the first table of de Boer's paper (2000).

In [4]:
# Defintions based on calculation functions from synthesise.cpp of de Boer's source code (2000)
def calculate_f1(p, h, r):
    f1 = ((-392+392*r)*pow(h, 2)+(596-668*r)*h-146+166*r)*pow(p, 2);
    f1 += ((348-348*r)*pow(h, 2)+(-494+606*r)*h+141-175*r)*p;
    f1 += ((340-72*r)*pow(h, 2)+(-796+108*r)*h+708-38*r);
    
    return f1;

def calculate_f2(p, h, r):
    f2 = ((-1200+1208*r)*pow(h, 2)+(1320-1328*r)*h+118-158*r)*pow(p, 2);
    f2 += ((1864-1488*r)*pow(h, 2)+(-2644+1510*r)*h-561+221*r)*p;
    f2 += ((-670+490*r)*pow(h, 2) + (1355-697*r)*h + 1517-117*r);
    
    return f2;

def calculate_f3(p, h, r):
    f3 = ((604-604*r)*pow(h, 2)+(1038-1178*r)*h+246+566*r)*pow(p, 2);
    f3 +=((-1150+1262*r)*pow(h, 2)+(-1443+1313*r)*h-317-483*r)*p;
    f3 +=((1130-836*r)*pow(h, 2)+(-315+44*r)*h+2427-127*r);
    
    return f3;

def calculate_f4(p, h, r):
    f4 = ((-1120+16*r)*pow(h, 2)+(1696-180*r)*h+500+522*r)*pow(p, 2);
    f4 +=((-140+240*r)*pow(h, 2)+(-578+214*r)*h-692-419*r)*p;
    f4 +=((1480-602*r)*pow(h, 2)+(-1220+289*r)*h+3678-178*r);
    
    return f4;


We now check if the above functions return the correct formant values by testing all of the values given in table 2 of de Boer (2000).

In [26]:
argument_sets = [[0, 0, 0],
                 [0, 0, 1],
                 [0.5, 0, 0],
                 [0.5, 0, 1],
                 [1, 0, 0],
                 [1, 0, 1],
                 [0, 0.5, 0],
                 [0, 0.5, 1],
                 [0.5, 0.5, 0],
                 [0.5, 0.5, 1],
                 [1, 0.5, 0],
                 [1, 0.5, 1],
                 [0, 1, 0],
                 [0, 1, 1],
                 [0.5, 1, 0],
                 [0.5, 1, 1],
                 [1, 1, 0],
                 [1, 1, 1]];

values = [[708, 1517, 2427, 3678],
          [670, 1400, 2300, 3500], 
          [742, 1266, 2330, 3457],
          [658, 1220, 2103, 3200],
          [703, 1074, 2356, 3486],
          [656, 1020, 2312, 3411],
          [395, 2027, 2552, 3438],
          [393, 1684, 2238, 3254],
          [399, 1438, 2118, 3197],
          [400, 1267, 2005, 2996],
          [430, 1088, 2142, 3490],
          [399, 829, 2143, 3490],
          [252, 2202, 3242, 3938],
          [250, 1878, 2323, 3447],
          [264, 1591, 2259, 3502],
          [276, 1319, 2082, 3118],
          [305, 1099, 2220, 3604],
          [276, 740, 2177, 3506]];

for i in range(0, len(argument_sets)):
    print(f"F1 calculation for {argument_sets[i]} equals expected {values[i][0]}: {calculate_f1(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2]) == values[i][0]} ({calculate_f1(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2])})");
    print(f"F2 calculation for {argument_sets[i]} equals expected {values[i][1]}: {calculate_f2(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2]) == values[i][1]} ({calculate_f2(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2])})");
    print(f"F3 calculation for {argument_sets[i]} equals expected {values[i][2]}: {calculate_f3(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2]) == values[i][2]} ({calculate_f3(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2])})");
    print(f"F4 calculation for {argument_sets[i]} equals expected {values[i][3]}: {calculate_f4(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2]) == values[i][3]} ({calculate_f4(argument_sets[i][0], argument_sets[i][1], argument_sets[i][2])})");
    print("---------------------------------------------------------------------");
    
    



F1 calculation for [0, 0, 0] equals expected 708: True (708)
F2 calculation for [0, 0, 0] equals expected 1517: True (1517)
F3 calculation for [0, 0, 0] equals expected 2427: True (2427)
F4 calculation for [0, 0, 0] equals expected 3678: True (3678)
---------------------------------------------------------------------
F1 calculation for [0, 0, 1] equals expected 670: True (670)
F2 calculation for [0, 0, 1] equals expected 1400: True (1400)
F3 calculation for [0, 0, 1] equals expected 2300: True (2300)
F4 calculation for [0, 0, 1] equals expected 3500: True (3500)
---------------------------------------------------------------------
F1 calculation for [0.5, 0, 0] equals expected 742: True (742.0)
F2 calculation for [0.5, 0, 0] equals expected 1266: True (1266.0)
F3 calculation for [0.5, 0, 0] equals expected 2330: True (2330.0)
F4 calculation for [0.5, 0, 0] equals expected 3457: True (3457.0)
---------------------------------------------------------------------
F1 calculation for [0.5,

We find that our functions perfectly match the expected output.

## Synthesizer functions with optional noise

From the above synthesizer equations, we can make the general needed synthesizer functions.
These functions will also allow for noise to be added so that the output frequency lies between:

$F^{out}_{i}(p, h, r) = F_{i}(p + \lambda, h + \lambda, r + \lambda)$

With $\lambda$ the shifting value. This random is picked uniformly between:

$\frac{-\psi}{2} \leq \lambda \leq \frac{\psi}{2}$ 

We can see that $\psi$ is thus an important value denoting the maximum allowed noise.